# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [8]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Carregar os dados 
df = pd.read_csv('previsao_de_renda.csv').dropna()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes: bool

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [10]:
reg1 = smf.ols('''np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + 
                C(tipo_renda, Treatment(reference="Assalariado")) + 
                C(educacao, Treatment(reference="Secundário")) + 
                C(estado_civil, Treatment(reference="Casado")) + 
                C(tipo_residencia, Treatment(reference="Casa")) + 
                idade + tempo_emprego + qt_pessoas_residencia''', data=df).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sun, 28 Dec 2025   Prob (F-statistic):               0.00
Time:                        21:29:05   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [11]:
reg2 = smf.ols('''np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + 
                C(tipo_renda, Treatment(reference="Assalariado")) + 
                C(educacao, Treatment(reference="Secundário")) + 
                C(estado_civil, Treatment(reference="Casado")) + 
                idade + tempo_emprego + qt_pessoas_residencia''', data=df).fit()
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Sun, 28 Dec 2025   Prob (F-statistic):               0.00
Time:                        21:30:06   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Divide os dados (75% treino, 25% teste)
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

In [17]:
# 4. Modelo Stepwise
# Seleção baseada nos p-values significativos da Tarefa 1
reg_stepwise = smf.ols('np.log(renda) ~ sexo + posse_de_imovel + idade + tempo_emprego', data=df_train).fit()

# Avaliação na base de teste
y_pred_stepwise = reg_stepwise.predict(df_test)
r2_stepwise = r2_score(np.log(df_test['renda']), y_pred_stepwise)

print(f"R² Stepwise (Teste): {r2_stepwise:.4f}")

R² Stepwise (Teste): 0.3563


In [12]:
reg_final = smf.ols('np.log(renda) ~ sexo + posse_de_imovel + idade + tempo_emprego', data=df).fit()
print(reg_final.summary())

print(f"\nR2 Inicial: {reg1.rsquared:.4f} | R2 Final: {reg_final.rsquared:.4f}")

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     1637.
Date:                Sun, 28 Dec 2025   Prob (F-statistic):               0.00
Time:                        21:30:23   Log-Likelihood:                -13685.
No. Observations:               12427   AIC:                         2.738e+04
Df Residuals:                   12422   BIC:                         2.742e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [18]:
# 5. Comparação de Parâmetros
print("Parâmetros Stepwise:")
print(reg_stepwise.params)

Parâmetros Stepwise:
Intercept                  7.279541
sexo[T.M]                  0.782641
posse_de_imovel[T.True]    0.098898
idade                      0.004326
tempo_emprego              0.060118
dtype: float64
